# Read the model and evaluate

import the necessary python modules

In [3]:
import tensorflow as tf
import tflite_runtime.interpreter as tflite
import numpy as np

2023-10-16 14:40:01.944389: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Create a tflite interpreter using the tflite model prepared by recipes.ipynb

In [4]:
model_path = "models/accu.tflite"
tflite_interpreter = tf.lite.Interpreter(
      model_path=model_path,
      experimental_op_resolver_type=tf.lite.experimental.OpResolverType.
      BUILTIN_REF,
  )
tflite_interpreter.allocate_tensors()

Check the input and output details
Note that the ouput tensor has index 6 !

In [5]:
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

In [6]:
print("input details:")
print(input_details)
print("")
print("First input tensor:  index: {:d}, shape: {} and dtype: {} : ".format(input_details[0]['index'],
                                                                           input_details[0]['shape'],
                                                                           input_details[0]['dtype']))
print("Second input tensor: index: {:d}, shape: {} and dtype: {}: ".format(input_details[1]['index'],
                                                                          input_details[1]['shape'],
                                                                          input_details[1]['dtype']))
#print(input_details[1]['index'],input_details[1]['shape'],input_details[1]['dtype'])

input details:
[{'name': 'serving_default_accumulate_add:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([1], dtype=int32), 'dtype': <class 'numpy.bool_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_accum_val:0', 'index': 1, 'shape': array([100], dtype=int32), 'shape_signature': array([100], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

First input tensor:  index: 0, shape: [1] and dtype: <class 'numpy.bool_'> : 
Second input tensor: index: 1, shape: [100] and dtype: <class 'numpy.float32'>: 


In [7]:
print("output details:")
print(output_details)

output details:
[{'name': 'StatefulPartitionedCall:0', 'index': 6, 'shape': array([100], dtype=int32), 'shape_signature': array([100], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [9]:
print("Ouput tensor: index: {:d}, shape: {} and dtype: {}".format(output_details[0]['index'],
                                                                  output_details[0]['shape'],
                                                                  output_details[0]['dtype']))

Ouput tensor: index: 6, shape: [100] and dtype: <class 'numpy.float32'>


Fill the input tensors and invoke the model with an array of 100 elements with value 15
These will be added to the accumulator because we set the first input tensor to True, which means adding

In [10]:
tflite_interpreter.set_tensor(0,[True])
tflite_interpreter.set_tensor(1,np.full((100,), 15.0, dtype=np.float32))
tflite_interpreter.invoke()

In [11]:
tflite_interpreter.get_tensor(output_details[0]['index'])

array([15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15.], dtype=float32)

Since the accumulator is set to all zeros initially and the second input tensor was set to 100 values of 15, we get those 100 values of 15

Next we subtract (first input tensor set to False) 100 values of 9.0

In [12]:
tflite_interpreter.set_tensor(0,[False])
tflite_interpreter.set_tensor(1,np.full((100,), 9.0, dtype=np.float32))
tflite_interpreter.invoke()

In [13]:
tflite_interpreter.get_tensor(output_details[0]['index'])

array([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
      dtype=float32)

and finally we add 100 values of 5

In [15]:
tflite_interpreter.set_tensor(0,[True])
tflite_interpreter.set_tensor(1,np.full((100,), 5.0, dtype=np.float32))
tflite_interpreter.invoke()

In [16]:
tflite_interpreter.get_tensor(output_details[0]['index'])

array([11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
       11., 11., 11., 11., 11., 11., 11., 11., 11.], dtype=float32)